<div style="display:flex; align-items:center; gap:10px;">
  <img src="ece_logo.png" width="198" height="91" alt="ECE logo" />
</div>

# **Rapport de projet – Analyse de données et apprentissage automatique**
## Réalisé par les étudiants de B3 Data & IA :
## _Kenza BELALOUI - Anis FETOUAB - Nathan BRUNET - Oleksandr KSHYVNYAK - Sirine BESSOUS_

### Ce projet a pour objectif de conduire un pipeline complet d’analyse de données, depuis le choix du dataset jusqu’à l’application d’un ou plusieurs modèles de machine learning.

# Partie 1 – Définition du sujet et choix du dataset
## 1.1 Thématique

Notre projet se situe dans le domaine du sport, plus précisément l'analyse de données dans le football.

L'objectif est de créer un modèle d'apprentissage automatique (machine learning) pour prédire si un joueur est un "Top Attaquant". Pour définir cette variable cible (TopAttacker), nous avons identifié les joueurs qui sont au-dessus de la médiane (la moyenne statistique) en GCA (Actions créant un but) et SCA (Actions créant un tir).

Ce sujet mélange nos deux passions : le football et l'application concrète de la data science. Le défi était d'utiliser des statistiques avancées (au-delà des simples buts) pour modéliser une notion qui est d'habitude subjective.

Ce projet sert concrètement au secteur sportif (clubs, médias) :

1.	Aide au recrutement : Les clubs peuvent utiliser le modèle pour repérer des talents sous-évalués, en se basant sur des métriques avancées comme les GCA et SCA.
2.	Analyse de performance : Les entraîneurs peuvent évaluer la contribution réelle d'un joueur à l'attaque, même s'il ne marque pas beaucoup.
3.	Médias et fans : Il fournit une base analytique pour comparer les joueurs et enrichir les débats.


## 1.2 Recherche et sélection du dataset
### Informations générales sur le dataset :

Nom du dataset :
Source et lien d’accès :
Auteur ou organisation :
Taille (nombre de lignes et de colonnes) : 2690 lignes et 97 colonnes
Format du fichier (CSV, JSON, Excel, etc.) : Fichier en .CSV

### Vérification de la qualité :
Le dataset est-il récent ?
Les variables sont-elles clairement nommées et documentées ?
Contient-il suffisamment de données (au moins plusieurs centaines de lignes) ?
Le dataset comporte-t-il une variable cible que vous pourrez prédire ou expliquer ?
Les données semblent-elles complètes et cohérentes ?

### Justification du choix :
Le fichier était idéal car il contenait les stats de GCA (Actions créant un But) et SCA (Actions créant un Tir). C'est crucial, car ces deux métriques nous ont permis de construire la variable cible (TopAttacker). Le reste des données (passes progressives, tirs, etc.) a servi de features (variables explicatives) pour entraîner le modèle.

#### Avantages

Richesse : Le dataset est hyper complet, avec des dizaines de métriques avancées comme PasProg (Passes Progressives) et CarProg (Portées Progressives), ce qui donne une description très fine du profil de chaque joueur.

- Comparabilité : Toutes les données sont ramenées "par 90 minutes" (90s), ce qui garantit une comparaison équitable entre les joueurs, peu importe leur temps de jeu total.

- Qualité : Les performances sont issues d'une compétition de haut niveau (Ligue des Champions), ce sont donc des données très pertinentes.

#### Limites et Difficultés

Nettoyage Technique : Le CSV a demandé un gros travail de preprocessing. On a dû corriger l'encodage (latin1) pour lire les noms des joueurs et surtout convertir beaucoup de colonnes en format numérique car le mélange de points et de virgules pour les décimales les rendait illisibles pour Python.

Données Manquantes : Certaines lignes avaient des valeurs nulles (NaN). On a été obligé de les supprimer, ce qui a réduit un peu notre échantillon de travail.

Déséquilibre des Classes : Le plus gros problème, c'est que nous avions peu de vrais "Top Attaquants" (classe 1) par rapport aux autres joueurs (classe 0). Ce dataset déséquilibré rend la tâche plus dure pour le modèle et nous oblige à utiliser des métriques plus solides que la simple accuracy.

## 1.3 Validation du dataset

### Résultats du Modèle KNN (Test Set)

| **Critères**      | **Questions**                                                    | **Réponses** | **Détail/Justification**
|:------------------|:-----------------------------------------------------------------|:-------------|--------------------------|
| **Pertinence**    | Le dataset permet-il de répondre à votre question de départ ?    | OUI          | **_A COMPLETER_**        |
| **Clarté**        | Les variables sont-elles bien nommées et compréhensibles ?       | NON          | **_A COMPLETER_**        |
| **Propreté**      | Les données semblent-elles utilisables sans nettoyage majeur ?   | NON          | **_A COMPLETER_**        |
| **Taille**        | Le dataset est-il d’une taille adaptée à votre analyse ?         | OUI          | **_A COMPLETER_**        |
| **Accessibilité** | Le format est-il compatible avec Python (CSV, XLSX) ?            | OUI          | **_A COMPLETER_**        |
| **Actualité**     | Les données sont-elles récentes ou encore valides ?              | OUI          | **_A COMPLETER_**        |

# Partie 2 – Exploration initiale des données
## 2.1 Chargement et aperçu du dataset (Importez le dataset dans un notebook Python à l’aide de pandas. )

#### Comment le dataset a-t-il été chargé ?

On a utilisé la bibliothèque pandas pour lire le fichier champions-league-2024-UTC.csv. La commande clé était :

df = pd.read_csv(csv_path, sep=';', decimal=',', index_col='Rk', encoding='latin1')

L'astuce a été de spécifier sep=';' (point-virgule) et decimal=',' (virgule) pour le format européen, et surtout encoding='latin1' pour éviter les erreurs de lecture de caractères spéciaux dans les noms de joueurs.

##### Nombre de lignes et de colonnes

Après le chargement, le DataFrame contenait un certain nombre de lignes (joueurs) et de colonnes (statistiques). Ça regroupe toutes les stats détaillées des joueurs de la Ligue des Champions 2024.

##### Principales variables (Features) clés

- Statistiques de but : Goals (buts marqués), Shots (tirs), SoT (tirs cadrés).
- Ratios : SoT% (pourcentage de tirs cadrés) et G/Sh, G/SoT (efficacité du tir).
- Métriques de Créativité (les plus importantes) : GCA, SCA, PasProg (Passes Progressives) et CarProg (Portées Progressives). C'est sur ces dernières qu'on a basé notre classification.
- Temps de jeu : 90s, qui est le temps total joué ramené à des matchs complets de 90 minutes.

##### Gestion des valeurs manquantes / incohérences

Nous avons dû gérer des valeurs manquantes (NaN) et des valeurs infinies (inf) dans certaines colonnes, notamment dans les ratios comme G/SoT. Une valeur devenait infinie quand, par exemple, un joueur n'avait aucun tir cadré (division par zéro).
Pour nettoyer ça et garantir la cohérence des données avant de modéliser, on a appliqué deux étapes :
<ol>
<li>Remplacer toutes les valeurs infinies par NaN (valeur manquante) : df.replace([np.inf, -np.inf], np.nan, inplace=True)</li>
<li>Remplacer ces NaN dans la colonne G/SoT par la médiane de la colonne (une valeur centrale) : df_filtered['G/SoT'] = df_filtered['G/SoT'].fillna(df_filtered['G/SoT'].median())</li>
</ol>

In [1]:
import pandas as pd

df = pd.read_csv('champions-league-2024-UTC.csv', sep=';')
df_attaquants = df[
    (df['Pos'].str.contains('FW', na=False)) & 
    (df['Min'] >= 300)
].copy()

print(f"Dataset initial : {df.shape}")
print(f"Dataset filtré (Attaquants) : {df_attaquants.shape}")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 7417: invalid continuation byte

## 2.2 Typologie des données Classification des variables

__Quantitatives continues :__ Ce sont principalement les ratios ou les données lissées par le temps de jeu, comme le SoT% (précision), G/Sh (efficacité du tir), ShoDist (distance moyenne de tir) et le temps de jeu 90s.

**Quantitatives discrètes :** Les décomptes bruts qui sont des nombres entiers, comme Goals, Shots, les métriques de créativité GCA et SCA, ainsi que PasProg et CarProg (passes et portées progressives).

**Qualitatives nominales :** Les identifiants comme le nom du joueur, son club ou sa position sur le terrain. Nous les avons exclues ou utilisées uniquement pour l'affichage, pas pour le modèle ML.

#### Variables les plus importantes pour l'analyse

Les features clés pour notre analyse sont celles liées à la création d'occasions : GCA, SCA, PasProg et CarProg. Ces variables sont essentielles car elles permettent de mesurer l'influence offensive globale d'un joueur, bien au-delà de ses buts personnels, et servent de base à notre classification.

#### Variable Cible :

Oui, l'objectif principal du projet est de prédire notre variable cible : TopAttacker (qui est binaire : 1 ou 0).

Définition technique : Nous avons créé cette cible en étiquetant un joueur 1 seulement si ses mesures de GCA_p90 (création de buts par 90 minutes) ET de SCA_p90 (création de tirs par 90 minutes) sont toutes deux au-dessus de la médiane des joueurs du dataset. En clair, on cible les joueurs qui excellent à la fois dans la phase de construction et dans l'avant-dernière passe.

# Partie 3 – Nettoyage et préparation du dataset

## 3.1 Gestion des valeurs manquantes

Après la conversion des données en numérique, les deux principales colonnes qui avaient des problèmes étaient G/SoT (Buts par Tir Cadré) et SoT% (Pourcentage de Tirs Cadrés). Ces NaN (valeurs manquantes) sont apparus souvent après qu'on ait transformé les valeurs infinies (inf) en NaN.

Notre stratégie a été l'imputation par la médiane :

```Python
df_numeric = df_numeric.fillna(df_numeric.median())
```
En résumé, nous avons remplacé chaque valeur manquante dans une colonne par la valeur médiane (la valeur centrale) de cette même colonne.

Nous avons choisi la médiane plutôt que la moyenne pour une raison technique : la médiane est moins sensible aux valeurs extrêmes (outliers).

Comme les ratios d'efficacité (comme G/SoT) peuvent avoir des valeurs très élevées ou très basses pour certains joueurs (ce qui fausserait la moyenne), la médiane donne une estimation plus robuste et représentative de la performance typique de l'ensemble des joueurs.

In [ ]:
cols_to_keep = [
    'Player', 'Squad', 'Age',       # Identité
    'Goals', 'Shots', 'SoT',        # Finition brute
    'SoT%', 'Assists',              # Précision et Altruisme
    'SCA', 'GCA',                   # Création (Shot/Goal Creating Actions)
    'TouAttPen',                    # Présence : Touches dans la surface de réparation
    'CarProg'                       # Percussion : Conduites de balle progressives
]

df_final = df_attaquants[cols_to_keep].copy()

rename_dict = {
    'Player': 'Joueur',
    'Squad': 'Equipe',
    'Goals': 'Buts',
    'Shots': 'Tirs_Total',
    'SoT': 'Tirs_Cadres',
    'SoT%': 'Tirs_Cadres_Pct',
    'Assists': 'Passes_Decisives',
    'SCA': 'Actions_Creation_Tir',
    'GCA': 'Actions_Creation_But',
    'TouAttPen': 'Touches_Surface',
    'CarProg': 'Percussions_Progressives'
}

df_final = df_final.rename(columns=rename_dict)
df_final = df_final.reset_index(drop=True)
display(df_final.head())

df_final.to_csv('top_attaquants_data.csv', index=False)

## 3.2 Détection et traitement des doublons

Techniquement, nous n'avons pas trouvé de doublons, car nous n'avons pas exécuté de vérification explicite (df.duplicated()) sur l'ensemble du DataFrame.

Toutefois, dans le contexte de ce projet, un tel contrôle n'a pas été jugé prioritaire. Notre dataset provient de statistiques de football très structurées, où chaque ligne représente une observation unique, indexée par le rang (Rk). Il est extrêmement improbable qu'un même joueur figure deux fois avec le même jeu de statistiques, car cela impliquerait une erreur dans la source de données elle-même.
**Aucun traitement spécifique** n'a été nécessaire.

Nous avons considéré que l'unicité des joueurs, garantie par la colonne _Player_ et l'index _Rk_, était suffisante.
Si nous avions trouvé des doublons, la stratégie standard dans le preprocessing des données aurait été de les identifier et de les supprimer immédiatement pour éviter de biaiser l'apprentissage du modèle :
Dans notre cas, nous nous sommes concentrés sur les étapes de nettoyage les plus cruciales pour le KNN : le traitement des valeurs manquantes et la normalisation des variables, qui étaient des problèmes bien plus critiques pour le bon fonctionnement de notre algorithme.

## 3.3 Détection des valeurs aberrantes

Des valeurs extrêmes ont été identifiées dans des colonnes basées sur des ratios, comme _G/SoT_ ou _SoT%_.

Ces valeurs ne sont pas considérées comme des erreurs de données, mais comme des cas particuliers qui reflètent la réalité sportive (par exemple, un joueur très efficace sur un très faible nombre de tirs).

La justification est double :
<ol>
<li>Elles représentent le profil de performance exceptionnelle que notre variable cible _(TopAttacker)_ cherche justement à isoler.</li>

<li>L'étape de normalisation (Standardisation) appliquée plus tard au dataset minimise l'influence disproportionnée de ces outliers sur le calcul des distances de l'algorithme KNN, rendant le modèle plus robuste.</li>
</ol>

Les seules corrections appliquées concernaient les valeurs infinies _(inf)_ qui rendaient le modèle inutilisable ; elles ont été traitées pour assurer la cohérence numérique du jeu de données.

## 3.4 Encodage et mise à l’échelle des variables

Aucun encodage n'a été nécessaire. Nous avons filtré les colonnes et utilisé uniquement les variables numériques du dataset, excluant les variables qualitatives (nom du joueur, club, etc.) de l'entraînement.

La méthode utilisée pour la mise à l'échelle est la Standardisation, implémentée avec la classe _StandardScaler_ de _scikit-learn_. Cette technique centre et réduit les variables pour qu'elles aient toutes une **moyenne de 0** et un **écart-type de 1**.

La standardisation est une étape obligatoire et cruciale pour l'algorithme des K-Plus Proches Voisins (KNN) que nous avons sélectionné :
<ol>
<li>Distance Euclidienne : Le KNN fonctionne en calculant la distance euclidienne entre les joueurs. Si nous n'harmonisons pas les échelles, les variables ayant une grande magnitude (comme _PasProg_, qui peut être un grand nombre) auraient un poids disproportionné sur la distance totale, faussant la notion de "proximité" des joueurs.</li>

<li>Comparabilité : Elle garantit que toutes les features (buts, tirs, créativité, etc.) sont mises sur un pied d'égalité, assurant ainsi la stabilité et la performance du modèle.</li>
</ol>

# Partie 4 – Analyse exploratoire et visualisations

#### 1. **Corrélations et Relations Observées :**
 Nous avons observé une corrélation positive marquée entre les métriques de création d'occasions (SCA - Actions de Création de Tir) et l'efficacité pure de la finition.
- Ca signifie qu'un joueur très impliqué dans la construction offensive (celui qui initie, dribble ou passe avant un tir) est statistiquement plus susceptible d'être performant à la finition. Cela tend à prouver que le Top Attaquant moderne ne se limite pas à marquer, il doit être un facteur d'influence global sur l'approche de la surface adverse.

#### 2. **Influence sur la Variable Cible :**
L'analyse de l'importance des features (post-modélisation) est très claire : ce ne sont pas uniquement les buts qui discriminent nos classes. Les variables qui décrivent la progression du jeu sont primordiales.
- Les métriques comme SCA, Percussions_Progressives (CarProg) et les passes progressives sont en tête de liste. Elles possèdent la plus forte capacité discriminante pour le modèle KNN, ce qui signifie que ces actions sont déterminantes pour classer un joueur dans la catégorie "Top Attaquant".

#### 3. **Hypothèses et Tendances Graphiques :**
Notre première hypothèse est que le succès d'un attaquant est multifactoriel. La classification ne dépend pas d'un seul pic de statistique, mais d'une répartition équilibrée de la performance sur plusieurs axes (finition, création, progression).
- Si on regarde nos tendances visuelles le Quadrant Plot (Création vs. Finition) confirme cette hypothèse en regroupant nos meilleurs joueurs dans le quadrant de "Haute Création et Haute Efficacité". Ils sont à la fois créateurs et finisseurs.

- Le Radar Plot renforce cette idée en montrant que le profil moyen des "Top Finisseurs" présente une enveloppe statistique plus large et homogène sur toutes les features comparées aux autres joueurs.